# XML Builder for OpR3a/b - V1.5

### READ ME! 
As the SWG needs to generate a large number of pointings for OpR3a. The following code is designed to take the BlankXML template from Cambridge and populate the XML with the various values needed for OpR3a and write out XML catalogs that are compatible with WASP/OCS.


Included is a GUI to set the appropriate PROGTEMP and OBSTEMP codes based on the programme and observation constraints you set.

To make use of this notebooks full functionality, you will need to install [IPyWidgets](http://ipywidgets.readthedocs.io/en/stable/user_install.html). Please note the full installation instructions, including the requirement of [node.js](https://nodejs.org/en/) being installed on your machine *and* the command to run the Jupyter Lab extension.
Note: This code *should* be compatible with both Python 2 and 3 (thanks to six.moves). However, this has not been tested.

### Changelog
* 1.3 - S3-ready, with updates to: moondist_min, sky_brightness and hour_angle_limits. Also added 30min OB PROGTEMP options.
* 1.4b - BETA: OBSTEMP entries are no longer hard-coded (PROGTEMP in-progress). Added plate selection and hour_angle_limit selection slider under new "misc options" cell. Added action-linked OB length dropdown selection.
* 1.5 - implementation of LIFU target generation, OBSTEMP radio buttons should now be ordered max -> min


### Contributors
* Kenneth Duncan (duncan@strw.leidenuniv.nl)
* Edouard Bernard
* David Murphy


In [1]:
#!/usr/bin/env python
import sys
import os.path
import numpy as np
#from six import moves
from six.moves import urllib
from ifu import lifu

import collections
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
import xml.dom.minidom
from xml.dom.minidom import Node

def remove_xml_declaration(xml_text):
    doc = xml.dom.minidom.parseString(xml_text)
    root = doc.documentElement
    xml_text_without_declaration = root.toxml(doc.encoding)
    return xml_text_without_declaration

def remove_empty_lines(xml_text):
    reparsed = xml.dom.minidom.parseString(xml_text)
    return '\n'.join([line for line in reparsed.toprettyxml(indent=' ').split('\n') if line.strip()])
import tempgui as g
from IPython.display import display



OBSTEMP: CAAEB
Max. Airmass: 1.3
PROGTEMP: 11331


____________
# <span style="color:dodgerblue">START HERE</span>

## Set PROGTEMP Code

__Run the cell below and then choose the desired programme constraints for this set of XML files.__ 

The corresponding PROGTEMP code is displayed in the lower right for reference. A variable, 'progtemp', is also set based on this code. If you change the buttons in this box, the variable will change accordingly.

To do:
- Extend OB Length and Exposure options to include all combinations being used in OpR3a

In [2]:
display(g.prog_controls)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oUmFkaW9CdXR0b25zKGRlc2NyaXB0aW9uPXUnSW5zdHJ1bWVudCBDb25maWd1cmF0aW9uOicsIGxheW91dD1MYXlvdXQod2lkdGg9dScxMDDigKY=


### Notes

- Conte(X)tual OB actions: This component is entirely optional - XMLChecking will not fail if there is no X value in the PROGTEMP.

|Survey | X | Action |
|:--------------|:------|:--------|
| StePS         |  Any  |Duplicate this OB the specified number of times (e.g. 11331.7 = duplicate this target configuration 7 times) |
| SCIP           | <4 | Duplicate this OB the specified number of times. OBs must be time-linked to ensure <72 hrs between first and last |
| IFUs (not LOFAR)|  3,5,7 | Initiate a dither sequence of 3, 5 or 7 pointings |
| LOFAR          |   Any  | Duplicate this OB the specified number of times (e.g 61331.7 = duplicate this target configuration 7 times) |

- Chain observations: If contextual action includes duplication of the OB (i.e. X > 1), setting 'Chain observations' == True will increase the (internal survey) priority on the subsequent OBs.

-----

## Set OBSTEMP Code

Run the cell below and then choose desired environmental constraints for this set of XML files. The corresponding OBSTEMP code is displayed in the lower right for reference. The variable 'obstemp' is also set based on this code. If you change the buttons in this box, the variable will change accordingly.

In [3]:
display(g.obs_controls)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oRmxvYXRTbGlkZXIodmFsdWU9MC45LCBkZXNjcmlwdGlvbj11J1NlZWluZyBNYXg6JywgbGF5b3V0PUxheW91dCh3aWR0aD11JzMwJScpLCDigKY=


To illustrate that these values are now stored:

In [4]:
print('The current PROGTEMP is: {0}{1}And the current OBSTEMP is: {2}'.format(g.progtemp, '\n', g.obstemp))

The current PROGTEMP is: 11331
And the current OBSTEMP is: CAAEB


## Select misc. options:


In [5]:
display(g.misc_controls)

VkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oVG9nZ2xlQnV0dG9ucyhkZXNjcmlwdGlvbj11J1BsYXRlOicsIG9wdGlvbnM9KCdQTEFURV9BJywgJ1BMQVRFX0InLCAnTElGVScpLCB2YWzigKY=


## Build OB XML file

First, we get the latest version of the blank XML Template:

In [6]:
try:
    template_path = "http://casu.ast.cam.ac.uk/~dmurphy/opr3/swg/resources/BlankXMLTemplate.xml"
    froot, fname = os.path.split(template_path)
    xml_template, info = urllib.request.urlretrieve(template_path, fname)
except:
    raise SystemExit("Could not download XML template")
    #xml_template = "/Users/david/Dropbox/CASU-WEAVE/ifu/tmp2/BlankXMLTemplate.xml"

Parse in the XML template ready to fill and print version number for reference.

In [7]:
try:
    dom = xml.dom.minidom.parse(xml_template)
except xml.parsers.expat.ExpatError:
    print("File {0} would not parse".format(xml_template))

root = dom.childNodes[0]

programme = root.childNodes[3]
observation = root.childNodes[5]

# Get the main nodes we are going to fill ready for convenience:
obsconstraints = dom.getElementsByTagName('obsconstraints')[0]
configure = dom.getElementsByTagName('configure')[0]
fields = dom.getElementsByTagName('fields')[0] 
field = fields.getElementsByTagName('field')[0]
spectrograph = dom.getElementsByTagName('spectrograph')[0]
surveys = observation.getElementsByTagName('surveys')[0]

In [8]:
print('XML Template Version: {0}'.format(dom.getElementsByTagName('root')[0].attributes['version'].value))

XML Template Version: 1.10


## Set additional properties

In [9]:
semester = 'S4'
ob_class = 'science' # Or calibration?
author = 'hess@astro.rug.nl'
cc_report = 'jfalcon@iac.es,isa@ugr.es,dmurphy@ast.cam.ac.uk'
report_verbosity = "0"

## Fill in fixed properties

With the OBSTEMP and PROGTEMP properties (and their constituent attributes) set, we now fill in the XML file with these properties under that assumption that we are creating a group of similar OBs. The cell below therefore does not need changed.

In [10]:
root.setAttribute('author', author)
root.setAttribute('cc_report', cc_report)
root.setAttribute('report_verbosity', report_verbosity)


observation.setAttribute('progtemp', value=g.progtemp)
observation.setAttribute('obs_type', value=g.obstype)
observation.setAttribute('ob_class', value=ob_class) 
observation.setAttribute('semester', value=semester)
if g.progtemp[0] in ['1','2','3']:
    observation.setAttribute('pa', value='0.0') #IFU module will write this for LIFU/mIFU

if g.progtemp.endswith('+'):
    observation.setAttribute('chained', value='True')
else:
    observation.setAttribute('chained', value='False')    
observation.setAttribute('obsgroup', value='')

redarm = spectrograph.getElementsByTagName('red_Arm')[0]
redarm.setAttribute('resolution', value=g.resolution)
redarm.setAttribute('binning_Y', value=g.I.value)
redarm.setAttribute('VPH', value=g.red_vph)

bluearm = spectrograph.getElementsByTagName('blue_Arm')[0]
bluearm.setAttribute('resolution', value=g.resolution)
bluearm.setAttribute('binning_Y', value=g.I.value)
bluearm.setAttribute('VPH', value=g.blue_vph)

obsconstraints.setAttribute('obstemp', value=g.obstemp)
obsconstraints.setAttribute('seeing_max', value=str(g.seeing.value))
obsconstraints.setAttribute('moondist_min', value=g.moon.label)
obsconstraints.setAttribute('transparency_min', value=g.transparency.label)
obsconstraints.setAttribute('elevation_min', value=g.elevation.label[:5])
obsconstraints.setAttribute('skybright_max', value=g.skybright.label)

ha_limits = configure.getElementsByTagName('hour_angle_limits')[0]
ha_limits.setAttribute('earliest', str(g.hour_angle_range.value[0]))
ha_limits.setAttribute('latest', str(g.hour_angle_range.value[1]))

configure.setAttribute('plate', g.plate.value)

Given the PROGTEMP, no create the correct number of exposures with the correct properties. The following cell should not be edited.

In [11]:
exposures = programme.getElementsByTagName('exposures')[0]
exp_list = exposures.getElementsByTagName('exposure')

science_exposure = exposures.childNodes[11].cloneNode(True)
exposures.removeChild(exposures.childNodes[11])
#print g.ORB
#nexp, texp, arm = g.orb_vals[g.ORB.index]
nexp, texp, arm = g.orb_vals[g.ORB.value]

for i in range(nexp):
    new_exposure = science_exposure.cloneNode(True)
    new_exposure.setAttribute('order', value=str(3+i))
    new_exposure.setAttribute('exp_time', value=str(texp))
    new_exposure.setAttribute('arm', value=arm)
    exposures.appendChild(new_exposure)


exp_list[-2].setAttribute('order', value=str(3+i+1))
exp_list[-1].setAttribute('order', value=str(3+i+1))

# Reload exposure list, sort and replace
exp_list = exposures.getElementsByTagName('exposure')
exp_list.sort(key=lambda x: int(x.attributes['order'].value))
exposures.childNodes = exp_list

__Note:__ If you change PROGTEMP and wish to regenerate the XML files, you __must__ rerun all of the above sections so that the Exposure properties are set correctly.


## Mimic Configure Information
The following attributes should be filled by Configure. For now we spoof the values in order to pass WASP requirements for OpR3. The values below are likely all suitable as they are, with the possible exception of the plate which we may want to change later when generating many fields.

In [12]:
num_sky_fibres = "10"
max_calibration = "25"
max_guide = "8"  
max_sky = "100"
maximum_gate_angle = "14.1"
config_file_version="0.0003"
configure_version = "1.0.0046"
plate_version="test_0004"
seed = "1"

if g.progtemp[0] in ['4','5','6']:
    max_guide = "1"

In [13]:
configure.setAttribute('num_sky_fibres', num_sky_fibres)
#configure.setAttribute('max_calibration', max_calibration) ##immutable, so doesn't really need to be assigned
configure.setAttribute('max_guide', max_guide)  ##changed in Template v1.05
#configure.setAttribute('max_sky', max_sky)  ##changed in Template v1.05
#configure.setAttribute('maximum_gate_angle', maximum_gate_angle) ##immutable, so doesn't really need to be assigned
configure.setAttribute('config_file_version', config_file_version) ##immutable, so doesn't really need to be assigned
configure.setAttribute('configure_version', configure_version)
configure.setAttribute('plate_version', plate_version) ##immutable, so doesn't really need to be assigned
configure.setAttribute('seed', seed)

In [14]:
conditions = configure.getElementsByTagName('conditions')[0]

conditions.setAttribute('epoch', '2015.0')
conditions.setAttribute('ha', '0')
conditions.setAttribute('relative_humidity', '0.2')
conditions.setAttribute('tlr', '0.0065')
conditions.setAttribute('temperature', '293.0')
conditions.setAttribute('pressure', '770.0')

## Fill in 'Surveys' information:
This is also likely to be the same for a block of OBs so initially I will just set an example and leave it for the user to edit as appropriate.

To do:
- Add 'targets_use' attribute that is also required by Configure

In [15]:
# Remove the existing blank survey node
surveys.removeChild(surveys.childNodes[1])

<DOM Element: survey at 0x7f19477b7638>

In [16]:
surveys_list = ['WA']
max_fibres = [603]
if g.progtemp[0] in ['1','2','3']:
    #this is MOS, so use the same surveys etc as in v1.4
    surveys_list = ['WEAVE_GA_LRhighl', 'WEAVE_WL', 'WEAVE_WQ']
    max_fibres = [610, 219, 184]


for ixs, survey in enumerate(surveys_list):
    new_survey = dom.createElement("survey")
    new_survey.setAttribute('name', value=str(survey))
    new_survey.setAttribute('priority', value=u'1.0')
    new_survey.setAttribute('max_fibres', value=u'{0}'.format(max_fibres[ixs]))
    surveys.appendChild(new_survey)

## Complete field-specific information and save (for MOS):

Finally, we need to fill in the properties that *are* specific to each field such as the Field coordinates, the OB name etc. Here I load in a list of fields as a catalog that contains the values i want to fill in.

### <span style="color:dodgerblue"> Modify these cells </span>

In [17]:
#field_list = Table.read('/Users/ken/surfdrive/Astro/WEAVE/OpR3/Fields/Candidate/deepfields_list_v2.txt', format='ascii.commented_header', header_start=5)
if g.progtemp[0] in ['1','2','3']:
    #this is MOS - load the StePS file
    field_list = Table.read('./StePSfields_list.txt',  format='ascii.ipac')
else:    
    field_list = []
    

In [18]:
#for i, row in enumerate(field_list):
for i, row in enumerate(field_list[:5]):
    if not g.progtemp[0] in ['1','2','3']:
        #ifu stuff, so don't do anything - just move on
        continue
    
    """
    MANUAL PARAMETERS
    (These values should be filled based on your input field list )
    Set observation parameters:    
    """ 
    ### Warning - When setting values from a table, be sure to wrap any floats/integers with str() ###
    
    observation.setAttribute('name', value=str(row['NAME'])) # OB Name
    
    field.setAttribute('RA_d', value=str(row['RA'])) # Field center RA in decimal degrees
    field.setAttribute('Dec_d', value=str(row['DEC'])) # Field center Dec in decimal degrees
    field.setAttribute('order', value='') # This attribute is not used for MOS (dithered IFU only)
    
    #field.setAttribute('pa', value='0.0') # Defined for IFU or set during configure?
    
    outxml = 'testob_{0}.xml'.format(row['NAME']) # Filename the new XML output
    
    """
    Now we parse the XML output using Edouard's functions to tidy up.
    """
    
    newxml = remove_empty_lines(dom.toprettyxml())
    finalxml = remove_xml_declaration(newxml)
    
    """
    Finally, the filled XML file is saved to the output file set above.
    """
    with open(outxml, 'w') as f:
        print "Writing to %s"%(outxml)
        f.write(finalxml)    

Writing to testob_1.xml
Writing to testob_2.xml
Writing to testob_3.xml
Writing to testob_4.xml
Writing to testob_5.xml


## Complete field-specific information and save (for IFU):

We now point to the input LIFU FITS catalogue to generate the targets.

The target elements will inherit *most* of the input catalogue spaxel data, so the input catalogue must specify each spaxel for each dither for each pointing.

The output filenames will be generated from the TARGID entries of the central spaxels

### <span style="color:dodgerblue"> Enter the path to your input FITS catalogue:</span>

In [19]:
input_cat = './WA_FITS_catalogue_180523-132206ex2.fits'
input_cat = './WA_FITS_catalogue_180608-143105ex2.fits'
input_cat = './WA_S4.fits'

In [20]:
if g.progtemp[0] in ['4','5','6']:
    #what resolution has the user specified?
    res = 'LR'
    if g.progtemp[0] != '4':
        res = 'HR'
    binning = g.progtemp[4]
    #instantiate the class
    ifu_data = lifu(input_cat,res=res,binning=binning)
    #analyse the input catalogue and generate the 
    centrals,groups = ifu_data.get_central_spaxels(dither_group=True)
    #connect the existing XML data to the lifu_data class
    ifu_data.ingest_xml(dom)    
    #generate all the OBs
    ifu_data.generate()

## Non-GUI OBSTEMP Codes:

As the environmental constraints may vary within a set of target fields of the same type, it may be desirable to set the OBSTEMP and corresponding properties for each source in the loop. In that case, the following function generates an OBSTEMP code given some constraints:

In [21]:
obstemp_source = g.getobstemp(seeing='1.1', moon='30', transparency='0.4', 
                              elevation='50.28', skybright='21.5') # Note the strings.

In [22]:
print(obstemp_source)

EEADB
